# Analysis of data produced by "time-dependent_fusion+ComPat_chi_normalised_by_grad_Te.ipynb"

The results are based on solving a diffusion equation for heat transport in a cylindrical approximation of a tokamak.

Here the interest is on the effect of varying parameters related to the induction of stochasticity onto a relatively simple model.

## A pretty standard set of imports

In [1]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100

## Read the data

The input parameters to the model that are used here are:

- s2, the randomness factor in the overshoot phase (here the same as s3)
- s3, the ramdomness factor in the post-overdhoot factor (here the same as s2)
- dt, the time-step used to solve the equation
- NITER, the number of time-steps, here determined by NITER * dt = 10s
- alpha, the exponential smoothing factor
- gamma, the dependence of the chi or flux on the ratio of the current temperature gradient to the steady state gradient (as an exponent of this ratio)

The output quantities we will analyze here are the mean and standard deviation of the core Te calculated over the last 50% of the iterations (i.e. the last 5 seconds of the simulation).

In [2]:
D = pd.read_csv('results_test.log')
print(D.columns)
D[['dt', 'NITER', 'alpha', 'beta', 'gamma', 's2', 's3', 'Te_0_mean_50%', 'Te_0_std_50%']]

Index(['N1', 'N2', 'N3', 's2', 's3', 'd2', 'd3', 'alpha', 'beta', 'gamma',
       'dt', 'NITER', 'flux_mean', 'flux_std', 'flux_ref', 'Te_0_mean_all',
       'Te_0_std_all', 'Te_0_mean_50%', 'Te_0_std_50%'],
      dtype='object')


,dt,NITER,alpha,beta,gamma,s2,s3,Te_0_mean_50%,Te_0_std_50%
0,0.10,100,0.0100,0.1,4.0,0.2000,0.2000,4.297601e+03,292.644716
1,0.10,100,0.0100,0.1,4.0,0.2000,0.2000,4.279885e+03,259.419679
2,0.10,100,0.0100,0.1,4.0,0.2000,0.2000,4.308608e+03,332.042786
3,0.10,100,0.1000,0.1,4.0,0.2000,0.2000,4.476138e+03,914.974059
4,0.10,100,0.0010,0.1,4.0,0.2000,0.2000,4.279409e+03,42.279203
5,0.01,1000,0.0010,0.1,4.0,0.2000,0.2000,4.270713e+03,63.687512
6,0.01,1000,0.0100,0.1,4.0,0.2000,0.2000,4.265578e+03,205.998984
7,0.01,1000,0.0010,0.1,4.0,0.2000,0.2000,4.263630e+03,75.033736
8,0.01,1000,0.0010,0.1,2.0,0.2000,0.2000,4.305890e+03,91.729407
9,0.01,1000,0.0010,0.1,2.0,0.2000,0.2000,4.246396e+03,85.566546


## Analysis
We expect the randomness factor, s2 = s3, to have a strong impact when it becomes large.

Below we see the impact on the mean core Te and note that the impact is large once the randomness factor is larger than about 0.2, but that there is also a strong impact on other parameters.

In [3]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[D.gamma==0.0].groupby(['dt', 'alpha']):
    g.plot(x='s2', y='Te_0_mean_50%', 
           style='o', logx=True, logy=True, ax=ax, 
           label='dt=%s, alpha=%s' % (np.ndarray.item(g.dt.unique()), np.ndarray.item(g.alpha.unique())), 
           title='Te_0 dependence on s2 by dt and alpha\ngamma=0.0')
plt.ylabel('Te_0_mean_50%') ; plt.legend(loc=0, ncol=5, fontsize=8);

This is even more obviously seen in the standard deviation of the core Te.

In [4]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[D.gamma==0.0].groupby(['dt', 'alpha']):
    g.plot(x='s2', y='Te_0_std_50%', 
           style='o', logx=True, logy=True, ax=ax, 
           label='dt=%s, alpha=%s' % (np.ndarray.item(g.dt.unique()), np.ndarray.item(g.alpha.unique())), 
           title='Te_0 dependence on s2 by dt and alpha\ngamma=0.0')
plt.ylabel('Te_0_std_50%') ; plt.legend(loc=0, ncol=5, fontsize=8);

We now zoom in on one choice of alpha (0.1) and dt (0.1) and see that the mean and standard deviation become comparable for a randomness factor of 0.5.

In [5]:
D[(D.alpha==0.1) & (D.dt==0.1)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                    style='o', logx=True, logy=True,
                                    title='Te_0 dependence on randomization level\nalpha=0.1, dt=0.1',
                                    figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Reducing dt from 0.1 to 0.01 delays this convergence of the mean and standard deviation until a randomness factor of 2.  This increases the cost of the computation, though, by a factor of 10.

In [6]:
D[(D.alpha==0.1) & (D.dt==0.01)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                      style='o', logx=True, logy=True,
                                      title='Te_0 dependence on randomization level\nalpha=0.1, dt=0.01',
                                      figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

If, instead of reducing dt, we reduce alpha from 0.1 to 0.01, we can delay the convergence until a randomness factor of 1, without increasing the cost of calculation.

In [7]:
D[(D.alpha==0.01) & (D.dt==0.1)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                      style='o', logx=True, logy=True,
                                      title='Te_0 dependence on randomization level\nalpha=0.01, dt=0.1',
                                      figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

With both dt and alpha set to 0.01, the overlap starts at the same point as for dt=0.01, alpha=0.1, i.e. for a randomness factor of 2, but the results at a randomness factor of 1 look better.

In [8]:
D[(D.alpha==0.01) & (D.dt==0.01)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                       style='o', logx=True, logy=True,
                                       title='Te_0 dependence on randomization level\nalpha=0.01, dt=0.01',
                                       figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Keeping alpha = 0.01 and dropping dt to 0.001 does not really give us an increase in the overlap randomness value.  Note that this increases the cost of the calculation by another factor of 10!

In [9]:
D[(D.alpha==0.01) & (D.dt==0.001)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                        style='o', logx=True, logy=True,
                                        title='Te_0 dependence on randomization level\nalpha=0.01, dt=0.001',
                                        figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Reducing alpha to 0.001 while keeping dt at 0.01 does improve things --- the overlap is now delayed until a randomness parameters of 5.  This improvement does not induce an additional cost above that which happenened when going from dt=0.1 to dt=0.01.

In [10]:
D[(D.alpha==0.001) & (D.dt==0.01)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                        style='o', logx=True, logy=True,
                                        title='Te_0 dependence on randomization level\nalpha=0.001, dt=0.01',
                                        figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Reducing both alpha and dt to 0.001 does not give a significant improvement over just reducing alpha.

In [11]:
D[(D.alpha==0.001) & (D.dt==0.001)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                         style='o', logx=True, logy=True,
                                         title='Te_0 dependence on randomization level\nalpha=0.001, dt=0.001',
                                         figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Here we select for a randomness factor of 1.0 and produce plots versus alpha separated by dt.

In [12]:
plt.figure(figsize=(18,6))
for i, dt in enumerate([0.1, 0.01, 0.001, 0.0001]):
    ax=plt.subplot(2,2,i+1)
    D[(D.s2==1.0) & (D.dt==dt)].plot(x='alpha', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                     style='o', logx=True, logy=False, 
                                     title='s2=s3 = 1.0, dt = %s' % (dt), ax=ax)
    plt.ylabel('$Te_0$ [$eV$]')
plt.suptitle('Te_0 dependence on alpha')
plt.subplots_adjust(hspace=0.4)

We see that there can be a problem for small alpha and large dt --- which is not surprising since the number of iterations is given by

NITER = 10.0 / dt

and we would like 

1 / alpha < NITER

or

alpha > dt / 10.0

Here we select for a randomness factor of 1.0 and produce plots versus alpha separated by dt, but filtering so that alpha >= dt/10.0

In [13]:
plt.figure(figsize=(18,6))
for i, dt in enumerate([0.1, 0.01, 0.001, 0.0001]):
    ax=plt.subplot(2,2,i+1)
    D[(D.s2==1.0) & (D.dt==dt) & (D.alpha >= D.dt/10.0)].plot(x='alpha', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                                     style='o', logx=True, logy=False, 
                                     title='s2=s3 = 1.0, dt = %s' % (dt), ax=ax)
    plt.ylabel('$Te_0$ [$eV$]')
plt.suptitle('Te_0 dependence on alpha, filtered so that alpha >= dt/10.0')
plt.subplots_adjust(hspace=0.5)

For a randomness factor of 0.2, all combinations of alpha and dt produce satisfactory results, though the results do improve as alpha is reduced (with filtering so that alpha >= dt/10.0).

In [14]:
D[(D.s2==0.2) & (D.alpha > D.dt/10.0)].plot(x='alpha', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                  style='o', logx=True, logy=False, 
                  title='Te_0 dependence on alpha, filtered so that alpha >= dt/10.0\ns2=s3 = 0.2',
                  figsize=(18,6))
plt.ylabel('$Te_0$ [$eV$]');

Now we concentrate on the standard deviation for the 0.2 randomness case, plotting versus alpha but breaking up the data by dt.  We can see that lower alpha is better, and that generally lower dt is better as well.

In [15]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[(D.s2==0.2) & (D.alpha > D.dt/10.0)].groupby('dt'):
    g.plot(x='alpha', y='Te_0_std_50%', 
           style='o', logx=True, logy=True, ax=ax, label='dt=%s' % (i),
           title='Te_0 dependence on alpha by dt, filtered so that alpha >= dt/10.0\ns2=s3 = 0.2')
plt.ylabel('Te_0_std_50%');

Showing only the gamma = 0.0 results, again shows the improvement that comes with reducing dt, even for the lowest alpha result.

In [16]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[(D.s2==0.2) & (D.gamma==0.0) & (D.alpha > D.dt/10.0)].groupby('dt'):
    g.plot(x='alpha', y='Te_0_std_50%', 
           style='o', logx=True, logy=True, ax=ax, label='dt=%s' % (i), 
           title='s2=%s, s3=%s gamma=%s, filtered so that alpha >= dt/10.0' % (g.s2.unique(), g.s3.unique(), g.gamma.unique()))
plt.ylabel('Te_0_std_50%');

Gamma, by itself, does not seem to have a big impact.

In [17]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[(D.s2==0.2) & (D.dt==0.001) & (D.alpha > D.dt/10.0)].groupby('alpha'):
    g.plot(x='gamma', y='Te_0_std_50%', 
           style='o', logx=False, logy=True, ax=ax, label='alpha=%s' % (i), 
           title='s2=%s, s3=%s dt=%s, filtered so that alpha >= dt/10.0' % (g.s2.unique(), g.s3.unique(), g.dt.unique()))
plt.ylabel('Te_0_std_50%');

We can also look at the mean and standard deviation of the fluxes, compared to the reference flux.

In [18]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[(D.gamma==0.0) & (D.s2<=10)].groupby(['dt', 'alpha']):
    g.plot(x='s2', y='flux_mean', 
           style='o', logx=True, logy=True, ax=ax, 
           label='dt=%s, alpha=%s' % (np.ndarray.item(g.dt.unique()), np.ndarray.item(g.alpha.unique())), 
           title='flux_mean dependence on s2 by dt and alpha\ns2=s3 = 0.2')
plt.ylim(1e3,1e6) ; plt.ylabel('flux_mean') ; plt.legend(loc=0, ncol=5, fontsize=8)
plt.plot([D.s2.min(), 10.0], [D.flux_ref.mean(), D.flux_ref.mean()], 'k');

In [19]:
fig, ax = plt.subplots(figsize=(18,6))
for i, g in D[(D.gamma==0.0) & (D.s2<=10)].groupby(['dt', 'alpha']):
    g.plot(x='s2', y='flux_std', 
           style='o', logx=True, logy=True, ax=ax, 
           label='dt=%s, alpha=%s' % (np.ndarray.item(g.dt.unique()), np.ndarray.item(g.alpha.unique())), 
           title='flux_std dependence on s2 by dt and alpha\ns2=s3 = 0.2')
plt.ylim(1e1,1e6) ; plt.ylabel('flux_std') ; plt.legend(loc=0, ncol=5, fontsize=8)
plt.plot([D.s2.min(), 10.0], [D.flux_ref.mean(), D.flux_ref.mean()], 'k');

Let us examine the cases with 
 - s2 == 0.2
 - dt >= 1e-3
 - Te_0_std_50% < 50 eV
 
We find that all such cases had an alpha of 0.001!

In [ ]:
D[(D.s2==0.2) & (D.dt >= 1e-3) & (D['Te_0_std_50%'] < 50.0)]\
  [['s2', 's3', 'alpha', 'beta', 'gamma', 'dt', 'NITER', 'Te_0_mean_50%', 'Te_0_std_50%']].describe().T

In [ ]:
D.describe().T

In [ ]:
pd.options.display.max_rows = 100 ; D